In [1]:
import mouseQPCRModelSelection 
import numpy as np
import time
import AssignGPGibbsSingleLoop
import GPflow
import numpy as np
import time
import cPickle as pickle
import assigngp_dense
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
Bpossible = np.linspace(2,70,3)
pt,Yall = mouseQPCRModelSelection.LoadMouseQPCRData(subsetSelection=0)
Y = Yall[:,0][None].T
print 'Y'
print Y.shape
strExp = 'MouseQPCR_Exp1'
m,mV = mouseQPCRModelSelection.InitModels(pt,Y)

Loaded data data/guo_ssData.p with nrowsXncols = (437, 2).
(437, 2)
(437,)
LoadMouseQPCRData output
(437, 2)
(437,)
Y
(437, 1)
Branching kernel =====================
unnamed.white.variance transform:+ve prior:None
[ 1.]
unnamed.branchkernelparam.Bv transform:(none) prior:None [FIXED]
[[ 1.]]
unnamed.branchkernelparam.kern.variance transform:+ve prior:None
[ 1.]
unnamed.branchkernelparam.kern.lengthscales transform:+ve prior:None
[ 1.]
Initialise models: MAP =====================

model.kern.white.variance transform:+ve prior:None
[ 1.]
model.kern.branchkernelparam.Bv transform:(none) prior:None [FIXED]
[[ 1.]]
model.kern.branchkernelparam.kern.variance transform:+ve prior:None
[ 1.]
model.kern.branchkernelparam.kern.lengthscales transform:+ve prior:None
[ 1.]
model.likelihood.variance transform:+ve prior:None
[ 1.]
Initialise models: VB =====================
compiling tensorflow function...
done


In [3]:
m.XExpanded.shape

(1311, 2)

In [4]:
Bpossible=None
strSaveState='test'
fSoftVBAssignment=False
fOptimizeHyperparameters = False
fReestimateMAPZ=False
numMAPsteps = 10
D=Y.shape[1]
print D

1


In [5]:
# Do the MAP solution and plot assignments
Kbranch = m.kern
pt = mV.t
b = 20

Bcrap = np.atleast_2d(b) # crappy branch point
t0 = time.time()

# reinitialise hyperparameters
Kbranch.white.variance = 1e-6
Kbranch.branchkernelparam.kern.lengthscales = 120 # 20 + (90. - b) / 2. # 65
Kbranch.branchkernelparam.kern.variance = 2 #0.0012 #  2.3158
m.likelihood.variance = 0.08
mV.likelihood.variance = m.likelihood.variance._array

# should recompute Kernel everytime we update kernel hyperparameters
m.CompileAssignmentProbability(fDebug=False,fMAP=True) 

# set branching point
m.kern.branchkernelparam.Bv = Bcrap 

print '============> B=' + str(m.kern.branchkernelparam.Bv._array.flatten())

# Random assignment for given branch point

np.random.seed(47)

randomAssignment = AssignGPGibbsSingleLoop.GetRandomInit(pt,Bcrap,m.indices)
print 'MAP assignment.'


(chainState, bestAssignment,_,condProbs) = \
    m.InferenceGibbsMAP(fReturnAssignmentHistory=True,fDebug=False,\
    maximumNumberOfSteps=numMAPsteps,\
    startingAssignment=list(randomAssignment))

totalTime = time.time()-t0
print 'MAP tooks ' + str(totalTime) + ' seconds.'    

============> B=[ 20.]
MAP assignment.
Performing MAP inference with B=[ 20.]


KeyboardInterrupt: 

In [ ]:
m

In [ ]:
mV

In [ ]:
# Plot MAP solution
assigngp_dense.PlotSample(D,m.XExpanded[bestAssignment, : ],3,Y,Bcrap,lw=5.,fs=30, \
                      mV=None, figsizeIn=(D*10,D*7),title='Posterior B=%.1f -loglik= %.2f VB= %.2f'%(b,0,0))

In [ ]:
Bpossible = np.linspace(22,50,5)
#Bpossible = [45]
logVBBound = []

Kbranch = m.kern
pt = mV.t

mV.kern.branchkernelparam.Bv.fixed = True # B not part of the state
mV.kern.white.variance = 1e-6
mV.kern.white.variance.fixed = True

stateSaved = mV.get_free_state().copy()

for ib,b in enumerate(Bpossible):
    t0 = time.time()

    mV.set_state(stateSaved)
    
    Bcrap = np.atleast_2d(b) # crappy branch point - remove?

    # reset branching allocations for before branching point
    bestAssignmentCensored = list(bestAssignment)
    for i,bi in enumerate(bestAssignment):
        if mV.X[bi,0] < b :
            # before branching point 
            bestAssignmentCensored[i] = m.indices[i][0]
        
    # Variational bound computation
    mV.kern.branchkernelparam.Bv = Bcrap
    mV._needs_recompile = True
    
    print 'Variational kernel branch value ' + str(mV.kern.branchkernelparam.Bv._array.flatten())
    # Set state for assignments
    
    mV.InitialisePhi(m.indices, bestAssignmentCensored, b, condProbs, fSoftAssignment = True, fSoftUni = True)

    mV.optimize(max_iters=100)
    
    VBbound = -mV.compute_log_likelihood()
    logVBBound.append(VBbound)

    assigngp_dense.PlotSample(D, mV.X[bestAssignmentCensored, : ],3,Y,Bcrap,lw=5.,fs=30, \
                          mV=mV, figsizeIn=(D*10,D*7), title='Posterior B=%.1f -loglik= %.2f VB= %.2f'%(b,0,VBbound))


    plt.figure()
    count=0
    for p, x, y in zip(mV.GetPhi(), mV.X[::3, 0], mV.Y):
        ps = p[count:count + 3]
        plt.scatter(x, y, 100, ps[2], vmin=0, vmax=1, cmap=plt.cm.viridis)
        count += 3
        
    totalTime = time.time()-t0
    print 'B=' + str(b) + '. VB bound=%.2f. Time=%g secs.'%(VBbound, totalTime)    

In [ ]:
nlogL = np.array(logVBBound)
iMin = np.argmin(nlogL)
plt.plot(Bpossible[:len(logVBBound)],nlogL, '-ob')
plt.title('-log marginal (vb) (b)')

v=plt.axis()
minB = Bpossible[iMin]
plt.plot([minB,minB],v[-2:],'--b')

print 'VB estimated B ' + str(minB) + ' with marginal likelihood ' + str(logVBBound[iMin])

In [ ]:
mV

In [ ]:
mV

In [ ]:
count=0
for p, x, y in zip(mV.GetPhi(), mV.X[::3, 0], mV.Y):
        ps = p[count:count + 3]
        plt.scatter(x, y, 100, ps[2], vmin=0, vmax=1, cmap=plt.cm.viridis)
        count += 3

In [ ]:
mV.GetPhi()[50:52,150:156].round()

In [ ]:
mV.InitialisePhi(m.indices, bestAssignmentCensored, b, condProbs, fSoftAssignment = True, fSoftUni = False)

In [ ]:
count=0
for p, x, y in zip(mV.GetPhi(), mV.X[::3, 0], mV.Y):
        ps = p[count:count + 3]
        plt.scatter(x, y, 100, ps[2], vmin=0, vmax=1, cmap=plt.cm.viridis)
        count += 3
plt.colorbar()

In [ ]:
Y.shape

In [ ]:
m.XExpanded.shape